Webscraping Bonus
1. Navigate to https://www.billboard.com/charts/hot-100/. Using BeautifulSoup, extract out the This Week, artist, song, Last Week, Peak Position, and Weeks on Chart values into a pandas DataFrame. Hint: The HTML for the number one ranked song is slightly different from that of the rest of the songs.

In [9]:
# Import libraries
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

In [10]:
# Get html of billboard hot 100 page
url = "https://www.billboard.com/charts/hot-100/"

# Make a request to get the page content
response = requests.get(url)
html = response.text

# Parse the HTML with BeautifulSoup
soup = BS(html, "html.parser")

print(type(response))
print(response.status_code)

<class 'requests.models.Response'>
200


In [11]:
# Billboard structures #1 differently
first_rank = soup.find('li', class_='o-chart-results-list__item').text.strip()
first_song = soup.find_all('h3', id='title-of-a-story')[2].text.strip()
first_artist = soup.find_all('span', class_='c-label')[1].text.strip()

# Stats for #1
stats = soup.find_all('span', class_='c-label')
lastweek = stats[2].text.strip()
peak = stats[3].text.strip()
weeks_chart = stats[4].text.strip()

# Store in lists
ranks = [first_rank]
songs = [first_song]
artists = [first_artist]
last_week = [lastweek]
peak_pos = [peak]
weeks_on_chart = [weeks_chart]

# Print to check
print("This Week:", ranks[0])
print("Song:", songs[0])
print("Artist:", artists[0])
print("Last Week:", last_week[0])
print("Peak Position:", peak_pos[0])
print("Weeks on Chart:", weeks_on_chart[0])

This Week: 1
Song: Golden
Artist: HUNTR/X: EJAE, Audrey Nuna & REI AMI
Last Week: 1
Peak Position: 1
Weeks on Chart: 15


In [15]:
data = []

# Find all song rows (each <ul> = one song entry)
chart_rows = soup.find_all("ul", class_="o-chart-results-list-row")

for row in chart_rows:
    # Song Rank
    rank_tag = row.find("span", class_="c-label")
    rank = rank_tag.text.strip() if rank_tag else "N/A"

    # Song title
    song_tag = row.find("h3", class_="c-title")
    song = song_tag.text.strip() if song_tag else "N/A"

    # Artist name
    artist_tag = row.find("span", class_="c-label a-no-trucate")
    if artist_tag:
        artist = artist_tag.text.strip()
    else:
        backup = row.find("ul", class_="lrv-a-unstyle-list")
        artist = backup.find("span").text.strip() if backup else "N/A"

    # Song stats (last week, peak, weeks on chart)
    stats = row.find_all("span", class_="c-label")
    if len(stats) >= 3:
        last_week = stats[-3].text.strip()
        peak = stats[-2].text.strip()
        weeks = stats[-1].text.strip()
    else:
        last_week = peak = weeks = "N/A"

    # Save into data list
    data.append({
        "This Week": rank,
        "Song": song,
        "Artist": artist,
        "Last Week": last_week,
        "Peak Position": peak,
        "Weeks on Chart": weeks
    })

# Build DataFrame
df = pd.DataFrame(data)

# Show summary info
print("Hot", len(df), "chart")
print("\nShape:", df.shape)
df.head()

Hot 100 chart

Shape: (100, 6)


,This Week,Song,Artist,Last Week,Peak Position,Weeks on Chart
0,1,Golden,"HUNTR/X: EJAE, Audrey Nuna & REI AMI",1,1,15
1,2,Ordinary,Alex Warren,2,1,34
2,3,Tit For Tat,Tate McRae,-,3,1
3,4,What I Want,Morgan Wallen Featuring Tate McRae,4,1,20
4,5,Daisies,Justin Bieber,8,2,12


2. After getting the code working for the current chart, navigate to last week's chart. Notice how the url for the page changes. Write a function which will, given a date, return a pandas DataFrame containing the Billboard chart data for that date.

3. Write a loop to retrieve the Billboard chart data for the last 10 weeks.